fair_facts_v2:

The Framework for Assessing Changes To Sea-level (FACTS) is an open-source modular, scalable, and extensive framework for global mean, regional, and extreme sea level projection that is designed to support the characterization of ambiguity in sea-level projections. It is designed so users can easily explore deep uncertainty by investigating the implications on GMSL, RSL, and ESL of different choices for different processes. Its modularity allows components to be represented by either simple or complex model. Because it is built upon the Radical-PILOT computing stack, different modules can be dispatched for execution on resources appropriate to their computational complexity.

FACTS is being developed by the Earth System Science & Policy Lab and the RADICAL Research Group at Rutgers University. FACTS is released under the MIT License.

Project Github Page: https://github.com/radical-collaboration/facts

In [1]:
#!/usr/bin/env python
# coding: utf-8

# fair_facts_v2:
# 
# The Framework for Assessing Changes To Sea-level (FACTS) is an open-source modular, scalable, and extensive framework for global mean, regional, and extreme sea level projection that is designed to support the characterization of ambiguity in sea-level projections. It is designed so users can easily explore deep uncertainty by investigating the implications on GMSL, RSL, and ESL of different choices for different processes. Its modularity allows components to be represented by either simple or complex model. Because it is built upon the Radical-PILOT computing stack, different modules can be dispatched for execution on resources appropriate to their computational complexity.
# 
# FACTS is being developed by the Earth System Science & Policy Lab and the RADICAL Research Group at Rutgers University. FACTS is released under the MIT License.
# 
# Project Github Page: https://github.com/radical-collaboration/facts

import logging
import time
import os
import shlex
import random
import subprocess
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# Setup logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def file_exists_and_has_content(filepath):
    """Check if file exists and has content"""
    path = Path(filepath)
    return path.is_file() and path.stat().st_size > 0


def setup_directories():
    """Ensure output directories exist"""
    os.makedirs('./data/output/fair', exist_ok=True)
    os.makedirs('./data/output/lws', exist_ok=True)
    os.makedirs('./data/output/sterodynamics', exist_ok=True)


def run_command(cmd, task_name=""):
    """Run a command synchronously and return results"""
    cmd_str = shlex.join(cmd)
    logger.info(f"[{task_name}] Executing: {cmd_str}")
    
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    if proc.stdout:
        logger.info(f"[{task_name}] stdout: {proc.stdout.decode()}")
    if proc.stderr:
        logger.warning(f"[{task_name}] stderr: {proc.stderr.decode()}")
    
    if proc.returncode != 0:
        raise RuntimeError(f"[{task_name}] Command failed with return code {proc.returncode}")
    
    return {
        'task': task_name,
        'command': cmd_str,
        'returncode': proc.returncode,
        'stdout': proc.stdout.decode(),
        'stderr': proc.stderr.decode()
    }


def fair_task():
    """FAIR temperature model task"""
    cmd = [
        '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
        '--bind', '/discover/nobackup/projects/sealevel/facts2.0/data/input:/input',
        '--bind', './data/output/fair:/output',
        '/discover/nobackup/projects/sealevel/facts2.0/containers/fair-temperature.sif',
        'fair-temperature',
        '--pipeline-id=1234',
        '--output-oceantemp-file=/output/oceantemp.nc',
        '--nsamps=20',
        '--output-ohc-file=/output/ohc.nc',
        '--output-gsat-file=/output/gsat.nc',
        '--output-climate-file=/output/climate.nc',
        '--rcmip-file=/input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv',
        '--param-file=/input/parameters/fair_ar6_climate_params_v4.0.nc'
    ]
    return run_command(cmd, "FAIR")


def lws_task():
    """Land Water Storage task - can run independently of FAIR"""
    cmd = [
        '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
        '--bind', '/discover/nobackup/projects/sealevel/facts2.0/data/input:/input',
        '--bind', './data/output/lws:/output',
        '/discover/nobackup/projects/sealevel/facts2.0/containers/ssp-landwaterstorage.sif',
        'ssp-landwaterstorage',
        '--pipeline-id=1234',
        '--nsamps=20',
        '--output-gslr-file=/output/gslr.nc',
        '--output-lslr-file=/output/lslr.nc',
        '--location-file=/input/location.lst',
        '--pophist-file=/input/UNWPP2012 population historical.csv',
        '--reservoir-file=/input/Chao2008 groundwater impoundment.csv',
        '--popscen-file=/input/ssp_iam_baseline_popscenarios2100.csv',
        '--gwd-file=/input/Konikow2011 GWD.csv',
        '--gwd-file=/input/Wada2012 GWD.csv',
        '--gwd-file=/input/Pokhrel2012 GWD.csv',
        '--fp-file=/input/REL_GROUNDWATER_NOMASK.nc'
    ]
    return run_command(cmd, "LWS")


def sterodynamics_task():
    """Sterodynamics task - depends on FAIR output"""
    cmd = [
        '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
        '--bind', '/discover/nobackup/projects/sealevel/facts2.0/data/input:/input',
        '--bind', './data/output/fair:/fair',
        '--bind', './data/output/sterodynamics:/output',
        '--nv',
        '/discover/nobackup/projects/sealevel/facts2.0/containers/tlm-sterodynamics.sif',
        'tlm-sterodynamics',
        '--pipeline-id=1234',
        '--scenario=ssp585',
        '--nsamps=20',
        '--model-dir=/input/cmip6/',
        '--location-file=/input/location.lst',
        '--output-lslr-file=/output/lslr.nc',
        '--output-gslr-file=/output/gslr.nc',
        '--expansion-coefficients-file=/input/scmpy2LM_RCMIP_CMIP6calpm_n18_expcoefs.nc',
        '--gsat-rmses-file=/input/scmpy2LM_RCMIP_CMIP6calpm_n17_gsat_rmse.nc',
        '--climate-data-file=/fair/climate.nc'
    ]
    return run_command(cmd, "Sterodynamics")


def total_task(component, name):
    """Facts total task - executes singularity command"""
    if component == 'all':
        filename = f'/mnt/total_out/totaled_output_all_{name}.nc'
        _filename = f'./data/output/totaled_output_all_{name}.nc'
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', './data/output:/mnt/total_in',
            '--bind', './data/output:/mnt/total_out',
            '/discover/nobackup/projects/sealevel/facts2.0/containers/sealevel-facts-total_latest-sandbox',
            'facts-total',
            f'--item=/mnt/total_out/lws/{name}.nc',
            f'--item=/mnt/total_out/sterodynamics/{name}.nc',
            '--pyear-start=2020',
            '--pyear-end=2150',
            '--pyear-step=10',
            f'--output-path={filename}'
        ]
    else:
        filename = f'/mnt/total_out/totaled_output_{component}_{name}.nc'
        _filename = f'./data/output/totaled_output_{component}_{name}.nc'
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', './data/output:/mnt/total_in',
            '--bind', './data/output:/mnt/total_out',
            '/discover/nobackup/projects/sealevel/facts2.0/containers/sealevel-facts-total_latest-sandbox',
            'facts-total',
            f'--item=/mnt/total_out/{component}/{name}.nc',
            '--pyear-start=2020',
            '--pyear-end=2150',
            '--pyear-step=10',
            f'--output-path={filename}'
        ]
    
    task_name = f"Total_{component}_{name}"
    result = run_command(cmd, task_name)
    
    # Verify that file was created
    if file_exists_and_has_content(_filename):
        logger.info(f"[{task_name}] Output successfully created: {_filename}")
    else:
        error_msg = f"Output unsuccessfully created: {_filename}"
        logger.error(f"[{task_name}] {error_msg}")
        raise RuntimeError(f"[{task_name}] Task failed: {error_msg}")
    
    result['component'] = component
    result['name'] = name
    return result


def run_modules():
    """Run the climate model modules"""
    transaction_id = random.randint(1, 1000)
    logger.info(f"Starting modules workflow: {transaction_id}")
    
    # Setup directories
    setup_directories()
    
    start_time = time.time()
    
    # Run FAIR and LWS in parallel (they are independent)
    with ThreadPoolExecutor(max_workers=2) as executor:
        fair_future = executor.submit(fair_task)
        lws_future = executor.submit(lws_task)
        
        # Wait for FAIR to complete first (sterodynamics depends on it)
        fair_result = fair_future.result()
        logger.info(f"FAIR task completed for pipeline {transaction_id}")
        
        # Get LWS result
        lws_result = lws_future.result()
        logger.info(f"LWS task completed for pipeline {transaction_id}")
    
    # Run sterodynamics (depends on FAIR output)
    sterodynamics_result = sterodynamics_task()
    logger.info(f"Sterodynamics task completed for pipeline {transaction_id}")
    
    elapsed_time = time.time() - start_time
    logger.info(f"Modules workflow {transaction_id} finished in {elapsed_time:.2f} seconds")
    
    results = {
        'fair': fair_result,
        'lws': lws_result,
        'sterodynamics': sterodynamics_result
    }
    
    logger.info(f"All modules completed successfully: {transaction_id}")
    logger.info(results)
    
    return results


def run_total():
    """Run the total aggregation workflow"""
    transaction_id = random.randint(1, 1000)
    logger.info(f"Starting total workflow: {transaction_id}")
    
    # Setup directories
    os.makedirs('./data/output', exist_ok=True)
    
    start_time = time.time()
    
    # Define all total tasks
    total_tasks = [
        ('lws', 'lslr'),
        ('lws', 'gslr'),
        ('sterodynamics', 'lslr'),
        ('sterodynamics', 'gslr'),
        ('all', 'lslr'),
        ('all', 'gslr'),
    ]
    
    results = []
    
    # Run all total tasks in parallel
    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = {
            executor.submit(total_task, component, name): (component, name)
            for component, name in total_tasks
        }
        
        for future in as_completed(futures):
            component, name = futures[future]
            try:
                result = future.result(timeout=60)  # 1 minute timeout per task
                results.append(result)
                logger.info(f"Total task {component}_{name} completed successfully")
            except Exception as e:
                logger.error(f"Total task {component}_{name} failed: {e}")
                results.append({
                    'component': component,
                    'name': name,
                    'error': str(e)
                })
    
    elapsed_time = time.time() - start_time
    logger.info(f"Total workflow {transaction_id} finished in {elapsed_time:.2f} seconds")
    logger.info(f"=========Total completed successfully=========: {transaction_id}")
    logger.info(results)
    
    return results


def main():
    """Main entry point"""
    try:
        # Run modules workflow
        modules_results = run_modules()
        
        # Run total workflow
        total_results = run_total()
        
        logger.info("All workflows completed successfully!")
        
        return {
            'modules': modules_results,
            'total': total_results
        }
        
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        logger.error("Manually clean up previous runs or change directory paths if a write error occurs")
        raise


if __name__ == '__main__':
    main()

2026-02-13 14:30:03,621 - __main__ - INFO - Starting modules workflow: 676
2026-02-13 14:30:03,624 - __main__ - INFO - [FAIR] Executing: /usr/local/other/singularity/4.0.3/bin/singularity exec --bind /discover/nobackup/projects/sealevel/facts2.0/data/input:/input --bind ./data/output/fair:/output /discover/nobackup/projects/sealevel/facts2.0/containers/fair-temperature.sif fair-temperature --pipeline-id=1234 --output-oceantemp-file=/output/oceantemp.nc --nsamps=20 --output-ohc-file=/output/ohc.nc --output-gsat-file=/output/gsat.nc --output-climate-file=/output/climate.nc --rcmip-file=/input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv --param-file=/input/parameters/fair_ar6_climate_params_v4.0.nc
2026-02-13 14:30:03,625 - __main__ - INFO - [LWS] Executing: /usr/local/other/singularity/4.0.3/bin/singularity exec --bind /discover/nobackup/projects/sealevel/facts2.0/data/input:/input --bind ./data/output/lws:/output /discover/nobackup/projects/sealevel/facts2.0/containers/ssp-landwaterst